# Goals

1. At first fix the vocabularies (unicode normalizer + also replace all the characters with one single one) then work on the dataset

* check if any shuffle has occured, causing mismatch

### resources

Total number of files to run on:

1. files in this folder : [annotated data](/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/annotated data)

2. these text files:

    a. [all data](/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/Input/all_data.txt)

    b. [train_data](/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/Input/train_data.txt)

    c. [test_data](/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/Input/train_data.txt)

3. this provided [json file](/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/annotations_from_json.txt) (done)

# final classes

* only allowed classes, I will use transfer learning on top of it


```
Label ID 	Label
0 	        O
1 	      B-PER
2 	      I-PER
3 	      B-ORG
4 	      I-ORG
5 	      B-LOC
6 	      I-LOC
```



* will work with this dataset later, at first get a dataset

# Setup

In [39]:
!wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
!pip install bnunicodenormalizer # unicode normalizer
!pip install swifter # for parallel apply in pandas
!pip install pandarallel # parallel apply in pandas

--2023-07-11 15:38:05--  https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7502 (7.3K) [text/plain]
Saving to: ‘conlleval.py.1’

conlleval.py.1      100%[===================>]   7.33K  --.-KB/s    in 0s      

2023-07-11 15:38:05 (77.5 MB/s) - ‘conlleval.py.1’ saved [7502/7502]



In [40]:
# basic libraries
import os
import numpy as np
import pandas as pd
import random
import gc

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# counter
from collections import Counter
# Huggingface libraries
from conlleval import evaluate
# unicode normalizer
from bnunicodenormalizer import Normalizer
bnorm=Normalizer()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Reading the dataset

In [41]:
datapath = "/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/annotations_from_json.txt"
with open(datapath) as f:
    lines = f.read().splitlines()
len(lines)

3545

## Extracting sentences and tags

In [42]:
for idx in range(len(lines)):
  demo = lines[idx]
  demo = demo.replace("[", "" )# at first tabs necessary to add count
  demo = demo.replace("]", "" )# at first tabs necessary to add count
  p = demo.split('"')
  print(p[1])
  print('\n')
  if idx == 5:
    break

অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ পরীক্ষার প্রশ্নপত্র ফাঁসের অভিযোগ উঠেছে।


ব্যাংকের চেয়ারম্যানও এ অভিযোগের সত্যতা স্বীকার করেছেন।


আজ শুক্রবার বিকেল চারটা থেকে পাঁচটা পর্যন্ত এক ঘণ্টার এ পরীক্ষা অনুষ্ঠিত হয়।


এ ছাড়া কয়েকটি পরীক্ষাকেন্দ্রে ছাত্রলীগের নেতা-কর্মীদের সহায়তায় মুঠোফোনে উত্তর সরবরাহের অভিযোগও পাওয়া গেছে।


ব্যাংকের চেয়ারম্যান ও ঢাকা বিশ্ববিদ্যালয়ের ইন্টারন্যাশনাল বিজনেস বিভাগের অধ্যাপক খন্দকার বজলুল হক প্রথম আলো ডটকমকে জানান, বিকেল তিনটা ৫০ মিনিটে তিনি এ ধরনের অভিযোগ পেয়েছেন।


তার হাতে ফাঁস হওয়া প্রশ্নের যে কাগজটি এসেছিল সেটির সঙ্গে প্রশ্নের মিল পাওয়া গেছে।




In [43]:
# taking all lines in a list
sentences = []
for idx in range(len(lines)):
  demo = lines[idx]
  demo = demo.replace("[", "" )# at first tabs necessary to add count
  demo = demo.replace("]", "" )# at first tabs necessary to add count
  p = demo.split('"')
  sentences.append(p[1])

In [44]:
sentences[6]

"তিনি বলেন, 'বেলা আড়াইটার দিকেই পরীক্ষার কেন্দ্রগুলোতে প্রশ্ন পাঠানো হয়।"

## separating tags

In [45]:
tags = []
for idx in range(len(lines)):
  Len = len(sentences[idx])
  demo = lines[idx]
  demo = demo.replace("[", "" )# at first tabs necessary to add count
  demo = demo.replace("]", "" )# at first tabs necessary to add count
  line = demo[1:Len+1]
  #print(line)
  demo = demo[Len+3:]
  demo = demo.replace(", ", "," )
  demo = demo.replace(" ", "" )
  demo = demo.replace('"', "" )
  #print(demo)
  p = demo.split(",")
  ############ adding all tags
  #print(p)
  #print("\n")
  tags.append(p)
  #if idx == 5:
    #break
print(lines[0])
print(tags[0])
len(tags)

["অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ পরীক্ষার প্রশ্নপত্র ফাঁসের অভিযোগ উঠেছে।", ["B-ORG", "L-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]]
['B-ORG', 'L-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


3545

### keeping only valid tags

In [46]:
# {0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}

# Flatten the list of lists into a single list
flattened_list = [item for sublist in tags for item in sublist]

# Get the unique values
unique_values = list(set(flattened_list))
print(unique_values)

['B-PERSON', 'U-GPE', 'U-ORG', 'I-ORG', 'B-LAW', 'B-ORG', 'I-LAW', 'I-PERSON', 'L-GPE', 'L-ORG', 'U-PERSON', 'L-LAW', 'O', 'I-GPE', 'U-DATE', 'L-PERSON', 'B-GPE']


In [47]:
toKeep = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
newTags = []
for i in range(len(tags)):
  demo = tags[i]
  for j in range(len(demo)):
    ###
    if demo[j] == 'B-PERSON':
      demo[j] = 'B-PER'
    elif demo[j] == 'I-PERSON':
      demo[j] = 'I-PER'


    ############### per done
    elif demo[j] == 'U-PERSON':
      demo[j] = 'B-PER'
    ##
    elif demo[j] == 'L-PERSON':
      demo[j] = 'I-PER'
    ############# org done
    elif demo[j] == 'U-ORG':
      demo[j] = 'B-ORG'
    ##
    elif demo[j] == 'L-ORG':
      demo[j] = 'I-ORG'
    ############### loc done
    elif demo[j] == 'B-GPE' or demo[j] =='U-GPE':
      demo[j] = 'B-LOC'
    ##
    elif demo[j] == 'I-GPE'or demo[j] =='I-GPE':
      demo[j] = 'I-LOC'


    ###########
    if demo[j] not in toKeep:
      demo[j] = 'O'
  #####
  newTags.append(demo)

# Flatten the list of lists into a single list
flattened_list = [item for sublist in newTags for item in sublist]

# Get the unique values
unique_values = list(set(flattened_list))
print(unique_values)

['I-PER', 'I-LOC', 'I-ORG', 'B-ORG', 'B-LOC', 'O', 'B-PER']


## normalizing sentences

In [48]:
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

In [49]:
# taking sentences and tags in a dataframe
df = pd.DataFrame(columns=['sentences', 'tags'])
df.sentences = sentences
df.tags = newTags#tags
df.head(10)

,sentences,tags
0,অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ প...,"[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O]"
1,ব্যাংকের চেয়ারম্যানও এ অভিযোগের সত্যতা স্বীকা...,"[O, O, O, O, O, O, O, O]"
2,আজ শুক্রবার বিকেল চারটা থেকে পাঁচটা পর্যন্ত এক...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,এ ছাড়া কয়েকটি পরীক্ষাকেন্দ্রে ছাত্রলীগের নেত...,"[O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O,..."
4,ব্যাংকের চেয়ারম্যান ও ঢাকা বিশ্ববিদ্যালয়ের ই...,"[O, O, O, B-ORG, I-ORG, B-ORG, I-ORG, I-ORG, B..."
5,তার হাতে ফাঁস হওয়া প্রশ্নের যে কাগজটি এসেছিল ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,"তিনি বলেন, 'বেলা আড়াইটার দিকেই পরীক্ষার কেন্দ...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,সেখান থেকে প্রশ্ন ফাঁস হতে পারে।',"[O, O, O, O, O, O, O, O]"
8,"পরীক্ষা বাতিল করা হবে কি না, এমন প্রশ্নের জবাব...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9,এ বিষয়ে পরিচালনা পর্ষদে আলোচনা করা হবে।,"[O, O, O, O, O, O, O, O]"


In [50]:
df.sentences= df.sentences.parallel_apply(lambda x:normalize(x))
df.head(10)

,sentences,tags
0,অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ পর...,"[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O]"
1,ব্যাংকের চেয়ারম্যানও এ অভিযোগের সত্যতা স্বীকার...,"[O, O, O, O, O, O, O, O]"
2,আজ শুক্রবার বিকেল চারটা থেকে পাঁচটা পর্যন্ত এক...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,এ ছাড়া কয়েকটি পরীক্ষাকেন্দ্রে ছাত্রলীগের নেতা-...,"[O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O,..."
4,ব্যাংকের চেয়ারম্যান ও ঢাকা বিশ্ববিদ্যালয়ের ইন্...,"[O, O, O, B-ORG, I-ORG, B-ORG, I-ORG, I-ORG, B..."
5,তার হাতে ফাঁস হওয়া প্রশ্নের যে কাগজটি এসেছিল স...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,"তিনি বলেন, 'বেলা আড়াইটার দিকেই পরীক্ষার কেন্দ্...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,সেখান থেকে প্রশ্ন ফাঁস হতে পারে।',"[O, O, O, O, O, O, O, O]"
8,"পরীক্ষা বাতিল করা হবে কি না, এমন প্রশ্নের জবাব...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9,এ বিষয়ে পরিচালনা পর্ষদে আলোচনা করা হবে।,"[O, O, O, O, O, O, O, O]"


## checking vocabulary of sentences

In [51]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentences"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}
vocab_all = extract_all_chars(df)

In [52]:
vocab_list = list(set(vocab_all["vocab"][0]))
print(len(vocab_list))
print(vocab_list) # from 140+ to 87, noice
toReplace = ["'",'।',',','(',' ','=','-','?','!',')','.',':','—','”',';']# can cause problems -> '০','ং','ঃ'
# ei tag gula pele 2 pasher sathe gap create kora lagbe! jate split korle sorasori ashe!

87
['ত', 'ল', ';', 'ঢ', 'র', 'থ', 'ঈ', 'ি', '\u200d', 'ছ', 'ঞ', '৫', ':', '১', 'ড', 'হ', 'ম', '৭', 'ঐ', 'া', 'ৈ', '৩', "'", 'আ', 'ঙ', 'ঋ', 'অ', '!', 'প', '৬', '৯', 'ও', 'এ', 'ণ', '.', '(', ',', '=', 'ে', 'ড়', 'ং', 'জ', 'ঢ়', ')', 'ই', 'ু', 'ভ', '।', 'ো', 'ৌ', 'ঝ', 'ফ', 'ৃ', 'শ', 'ৎ', 'ঃ', 'গ', '”', 'ী', 'ক', 'খ', 'ব', 'ঁ', 'ষ', 'দ', 'ঠ', 'ন', 'ঊ', 'ধ', '৪', 'য়', '২', '্', 'ঘ', '?', 'উ', 'য', 'ঔ', 'ট', '—', 'ূ', '-', ' ', 'চ', 'স', '০', '৮']


In [53]:
######### replacing erroreneous punctuation with a common token
########## I can use this special token : "PUNCT"
#df['sentences'] = df['sentences'].str.replace("[।,)(=?!'.:—”;-]", ' <PUNCT> ')# # space added before and after punct so that I can split tokens
#df['sentences'] = df['sentences'].str.replace('#', ' <PUNCT> ')# regex wasn't handling it for some reason
# why the thing doesn't work for #


# I preserved spaces as it will be used to split!

In [54]:
#df['sentences'] = df['sentences'].str.replace("[।,)(=?!'.:—”;-]", ' । ')# # space added before and after punct so that I can split tokens
df['sentences'] = df['sentences'].str.replace('#', ' # ')
df['sentences'] = df['sentences'].str.replace('।', ' । ')
df['sentences'] = df['sentences'].str.replace(',', ' , ')
df['sentences'] = df['sentences'].str.replace(')', ' ) ')
df['sentences'] = df['sentences'].str.replace('(', ' ( ')
df['sentences'] = df['sentences'].str.replace('=', ' = ')
df['sentences'] = df['sentences'].str.replace('?', ' ? ')
df['sentences'] = df['sentences'].str.replace('!', ' ! ')
df['sentences'] = df['sentences'].str.replace("'", " ' ")
df['sentences'] = df['sentences'].str.replace('.', ' . ')
df['sentences'] = df['sentences'].str.replace(':', ' : ')
df['sentences'] = df['sentences'].str.replace('—', ' — ')
df['sentences'] = df['sentences'].str.replace('”', ' ” ')
df['sentences'] = df['sentences'].str.replace(';', ' ; ')
df['sentences'] = df['sentences'].str.replace('-', ' - ')
df['sentences'] = df['sentences'].str.replace('#', ' # ')

<ipython-input-54-68a3d8c95d3d>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['sentences'] = df['sentences'].str.replace(')', ' ) ')
<ipython-input-54-68a3d8c95d3d>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['sentences'] = df['sentences'].str.replace('(', ' ( ')
<ipython-input-54-68a3d8c95d3d>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['sentences'] = df['sentences'].str.replace('?', ' ? ')
<ipython-input-54-68a3d8c95d3d>:11: FutureWarning: The default value of regex will change from True to False 

In [55]:
df.head(10)

,sentences,tags
0,অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ পর...,"[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O]"
1,ব্যাংকের চেয়ারম্যানও এ অভিযোগের সত্যতা স্বীকার...,"[O, O, O, O, O, O, O, O]"
2,আজ শুক্রবার বিকেল চারটা থেকে পাঁচটা পর্যন্ত এক...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,এ ছাড়া কয়েকটি পরীক্ষাকেন্দ্রে ছাত্রলীগের নেতা ...,"[O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O,..."
4,ব্যাংকের চেয়ারম্যান ও ঢাকা বিশ্ববিদ্যালয়ের ইন্...,"[O, O, O, B-ORG, I-ORG, B-ORG, I-ORG, I-ORG, B..."
5,তার হাতে ফাঁস হওয়া প্রশ্নের যে কাগজটি এসেছিল স...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,"তিনি বলেন , ' বেলা আড়াইটার দিকেই পরীক্ষার কে...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,সেখান থেকে প্রশ্ন ফাঁস হতে পারে । ',"[O, O, O, O, O, O, O, O]"
8,"পরীক্ষা বাতিল করা হবে কি না , এমন প্রশ্নের জব...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9,এ বিষয়ে পরিচালনা পর্ষদে আলোচনা করা হবে ।,"[O, O, O, O, O, O, O, O]"


* all punct are "o" here, so no problem I hope

## Extracting tokens

In [56]:
prevLines = lines.copy()

# checking
j = 7
print(sentences[j])
print(prevLines[j])

সেখান থেকে প্রশ্ন ফাঁস হতে পারে।'
["সেখান থেকে প্রশ্ন ফাঁস হতে পারে।'", ["O", "O", "O", "O", "O", "O", "O", "O"]]


In [57]:
lines = df["sentences"].tolist()
print(len(lines))
lines[7]

3545


"সেখান থেকে প্রশ্ন ফাঁস হতে পারে ।  ' "

* need to get rid of spaces between 2 <punc> tokens

In [58]:
tokens = []
for idx in range(len(lines)):
  #print(sentences[idx])
  line = lines[idx]
  line = line.split(" ") # there will be some anomaly, I will have to separate them again!
  token = []
  for j in range(len(line)):
    tok = line[j]
    if tok != '':
      token.append(tok)
  tokens.append(token)
  #print(token)
  #print("\n")

  #if idx == 20:
    #break
len(tokens)
# I want every punctuation to be separate tokens as given in the label

3545

In [59]:
## removing empty strings
#tokens = [token for token in tokens if token != '']

### verifying if number of tags and tokens are same

In [60]:
tags = df["tags"].tolist()
print(len(tags),len(tags[0]))
tags[0]

3545 12


['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [61]:
# so that multiple occurance cannot happen
def reducing_mulOcc(text):
    processed_text = ""
    previous_char = ""

    for char in text:
        if char == "\t" or char == " ":
            if char != previous_char:
                processed_text += char
        else:
            processed_text += char

        previous_char = char
    return processed_text

In [62]:
misIDX = []
c = 0
for j in range(len(tags)):
  a = len(tags[j])
  b = len(tokens[j])
  if a!=b:
    misIDX.append(j)
    if b<a:
      c+=1
print(len(misIDX))
c

200


63

* lowest till yet, 403 defectives!

* so still 714 tag token pair didn't match

* so the major problem for now is that token number is lower than tag as expected.


* If I can't solve this sooner, I should drop the excess indexes, focus on model. handle them later

* problem regarding #, why isn't it being handled!

* i wasn't handled

later check it in kaggle, to see if it has something with the hardware



```
জাতীয় অর্থোপেডিক হাসপাতাল ও পুনর্বাসন কেন্দ্রে (পঙ্গু হাসপাতাল) কার্যাদেশ পাওয়ার তিন দিনের মধ্যে যন্ত্রপাতি সরবরাহ করে তৌফিকা এনার্জি লিমিটেড নামের একটি প্রতিষ্ঠান।
['জাতীয়', 'অর্থোপেডিক', 'হাসপাতাল', 'ও', 'পুনর্বাসন', 'কেন্দ্রে', '(পঙ্গু', 'হাসপাতাল)', 'কার্যাদেশ', 'পাওয়ার', 'তিন', 'দিনের', 'মধ্যে', 'যন্ত্রপাতি', 'সরবরাহ', 'করে', 'তৌফিকা', 'এনার্জি', 'লিমিটেড', 'নামের', 'একটি', 'প্রতিষ্ঠান', '<PUNCT>']
['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'L-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'L-ORG', 'O', 'O', 'O', 'O']
23 25
```



In [63]:
misIDX[-2]

3494

In [64]:
for idx in misIDX[-20:]:
  print(sentences[idx])
  print(tokens[idx])
  print(tags[idx])
  print(len(tokens[idx]),len(tags[idx]))
  print("\n")

তা তখন ইংল্যান্ড এবং ভারতীয় উপমহাদেশ, দু'অঞ্চলেই সাড়া জাগিয়ে দিয়েছিল তুমুলভাবে।
['তা', 'তখন', 'ইংল্যান্ড', 'এবং', 'ভারতীয়', 'উপমহাদেশ', ',', 'দু', "'", 'অঞ্চলেই', 'সাড়া', 'জাগিয়ে', 'দিয়েছিল', 'তুমুলভাবে', '।']
['O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
15 13


১৯০৬ সালের ২০ আগস্ট তার মৃত্যুর পর শিক্ষা প্রতিষ্ঠানটি বন্ধ হয়ে গেলেও দু'বছর পর তা পুনরায় চালু করা হয়।
['১৯০৬', 'সালের', '২০', 'আগস্ট', 'তার', 'মৃত্যুর', 'পর', 'শিক্ষা', 'প্রতিষ্ঠানটি', 'বন্ধ', 'হয়ে', 'গেলেও', 'দু', "'", 'বছর', 'পর', 'তা', 'পুনরায়', 'চালু', 'করা', 'হয়', '।']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
22 20


বিলেত থেকে ফেরত এসে তিনি যখন দেখলেন, উপমহাদেশের ছাত্র সমাজ তাদের অবসর সময়ে দেশসেবায় নিজেদের বিলিয়ে দিচ্ছে; তারা নারীজাতির শিক্ষা, জনহিতকর কাজে নিয়োজিত- তখন তার মনে আশার দানা বাঁধল।
['বিলেত', 'থেকে', 'ফেরত', 'এসে', 'তিনি', 'যখন', 'দেখলেন', ',', 'উপমহাদেশের', 'ছাত্র', 'সমাজ', 'তাদের', 'অবসর', 'সময়ে', 'দেশসেবায়', 'নিজেদের', 

* I think tokenization is good enough for now, after running baseline model, I need to check the data, the labelling seems quite problematic

* also my job is to only select person, so other tags need to be converted to "O"

### dropping mismatched examples

In [65]:
finTokens = []
finTags   = []
finSentences = []
for i in range(len(sentences)):
  if i not in misIDX:
    finSentences.append(sentences[i])
    finTokens.append(tokens[i])
    finTags.append(tags[i])
len(finTags),len(finTokens),len(finSentences)

(3345, 3345, 3345)

In [66]:
# checking if any mismatch remains
misIDX = []
c = 0
for j in range(len(finTags)):
  a = len(finTags[j])
  b = len(finTokens[j])
  if a!=b:
    misIDX.append(j)
    if b<a:
      c+=1
print(len(misIDX))
c
# nice, no mismatch in token-tag pairs

0


0

* inspect some random token-tag pairs, to see if everything ok

In [67]:
start = 1  # Start of the range
end = len(finTags)  # End of the range
count = 20  # Number of random integers to choose

random_integers = random.sample(range(start, end + 1), count)

In [68]:
for idx in random_integers:
  print(sentences[idx])
  print(tokens[idx])
  print(tags[idx])
  print(len(tokens[idx]),len(tags[idx]))
  print("\n")

স্নায়ুযুদ্ধের সময় সোভিয়েত ইউনিয়নের নেতৃত্বে সমাজতান্ত্রিক বিশ্ব ও আমেরিকার নেতৃত্বে পুঁজিবাদী বিশ্ব মোটামুটি পৃথিবীর সব দেশকেই নিজেদের বলয়ে আনতে চেষ্টা করছিল।
['স্নায়ুযুদ্ধের', 'সময়', 'সোভিয়েত', 'ইউনিয়নের', 'নেতৃত্বে', 'সমাজতান্ত্রিক', 'বিশ্ব', 'ও', 'আমেরিকার', 'নেতৃত্বে', 'পুঁজিবাদী', 'বিশ্ব', 'মোটামুটি', 'পৃথিবীর', 'সব', 'দেশকেই', 'নিজেদের', 'বলয়ে', 'আনতে', 'চেষ্টা', 'করছিল', '।']
['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
22 22


গত বুধবার নির্বাচন কমিশন ৬ জুলাই নির্বাচনের দিন ধার্য করে তফসিল ঘোষণা করে।
['গত', 'বুধবার', 'নির্বাচন', 'কমিশন', '৬', 'জুলাই', 'নির্বাচনের', 'দিন', 'ধার্য', 'করে', 'তফসিল', 'ঘোষণা', 'করে', '।']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
14 14


ইশতিয়াকের বাবা শাহ আলমের পেশা ঠিকাদারি।
['ইশতিয়াকের', 'বাবা', 'শাহ', 'আলমের', 'পেশা', 'ঠিকাদারি', '।']
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O']
7 7


 ডা. ফারিয়া আফসানা, হরমোন ও ডায়াবেটিস বিভাগ বারডেম হাসপাতাল।


In [69]:
gc.collect()

18

## Making only person identifying tokens

In [70]:
df2 = pd.DataFrame(columns=['sentences', 'tokens','tags'])
df2.sentences = finSentences
df2.tokens = finTokens
df2.tags = finTags

df2.head(20)

,sentences,tokens,tags
0,অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ প...,"[অগ্রণী, ব্যাংকের, জ্যেষ্ঠ, কর্মকর্তা, পদে, নি...","[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O]"
1,ব্যাংকের চেয়ারম্যানও এ অভিযোগের সত্যতা স্বীকা...,"[ব্যাংকের, চেয়ারম্যানও, এ, অভিযোগের, সত্যতা, স...","[O, O, O, O, O, O, O, O]"
2,আজ শুক্রবার বিকেল চারটা থেকে পাঁচটা পর্যন্ত এক...,"[আজ, শুক্রবার, বিকেল, চারটা, থেকে, পাঁচটা, পর্...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,এ ছাড়া কয়েকটি পরীক্ষাকেন্দ্রে ছাত্রলীগের নেত...,"[এ, ছাড়া, কয়েকটি, পরীক্ষাকেন্দ্রে, ছাত্রলীগের,...","[O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O,..."
4,ব্যাংকের চেয়ারম্যান ও ঢাকা বিশ্ববিদ্যালয়ের ই...,"[ব্যাংকের, চেয়ারম্যান, ও, ঢাকা, বিশ্ববিদ্যালয়ে...","[O, O, O, B-ORG, I-ORG, B-ORG, I-ORG, I-ORG, B..."
5,তার হাতে ফাঁস হওয়া প্রশ্নের যে কাগজটি এসেছিল ...,"[তার, হাতে, ফাঁস, হওয়া, প্রশ্নের, যে, কাগজটি, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,"তিনি বলেন, 'বেলা আড়াইটার দিকেই পরীক্ষার কেন্দ...","[তিনি, বলেন, ,, ', বেলা, আড়াইটার, দিকেই, পরীক্...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,সেখান থেকে প্রশ্ন ফাঁস হতে পারে।',"[সেখান, থেকে, প্রশ্ন, ফাঁস, হতে, পারে, ।, ']","[O, O, O, O, O, O, O, O]"
8,"পরীক্ষা বাতিল করা হবে কি না, এমন প্রশ্নের জবাব...","[পরীক্ষা, বাতিল, করা, হবে, কি, না, ,, এমন, প্র...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9,এ বিষয়ে পরিচালনা পর্ষদে আলোচনা করা হবে।,"[এ, বিষয়ে, পরিচালনা, পর্ষদে, আলোচনা, করা, হবে, ।]","[O, O, O, O, O, O, O, O]"


In [71]:
########### take this to csv file for later usage
#savepath = "/content/drive/MyDrive/Hishab_assessment/Ds2_multiclass/"
#df2.to_csv('ds2_multiclass_clean_new.csv',index=False)#savepath+'ds2_multiclass_clean.csv'

In [72]:
# identifying unique values
unique_values = list(set([item for sublist in finTags for item in sublist]))
print(len(unique_values))
print(unique_values)
# among these I need only 'I-PERSON', 'L-PERSON', 'B-PERSON' == "PERSON", others = "O"

7
['I-PER', 'I-LOC', 'I-ORG', 'B-ORG', 'B-LOC', 'O', 'B-PER']


### Insights 1

* the model should extract person's name, not the tag only

* so the problem can be a binary classification problem, and if-

    * pred == yes, output == that token

    * else, output == no

I can train for named entity recognition and post process to give the name, I think that will do for now!

* very low number of methods have yes count, so need to do something like stratified k folding

In [73]:
newTags = []
yesIDX = []
for idx in range(len(finTags)):
  c = 0
  sublist = finTags[idx]
  sub = []
  for idx in range(len(sublist)):
    item = sublist[idx]
    if item == 'I-PER' or item =='B-PER':
      item = "yes"
      c = 1
    else:
      item = "no"
    sub.append(item)
  newTags.append(sub)
  if c==1:
    yesIDX.append(idx)
len(newTags),len(yesIDX)

(3345, 1103)

In [74]:
df3 = pd.DataFrame(columns=['sentences', 'tokens','tags'])
df3.sentences = finSentences
df3.tokens = finTokens
df3.tags = newTags

df3.head(20)

,sentences,tokens,tags
0,অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ প...,"[অগ্রণী, ব্যাংকের, জ্যেষ্ঠ, কর্মকর্তা, পদে, নি...","[no, no, no, no, no, no, no, no, no, no, no, no]"
1,ব্যাংকের চেয়ারম্যানও এ অভিযোগের সত্যতা স্বীকা...,"[ব্যাংকের, চেয়ারম্যানও, এ, অভিযোগের, সত্যতা, স...","[no, no, no, no, no, no, no, no]"
2,আজ শুক্রবার বিকেল চারটা থেকে পাঁচটা পর্যন্ত এক...,"[আজ, শুক্রবার, বিকেল, চারটা, থেকে, পাঁচটা, পর্...","[no, no, no, no, no, no, no, no, no, no, no, n..."
3,এ ছাড়া কয়েকটি পরীক্ষাকেন্দ্রে ছাত্রলীগের নেত...,"[এ, ছাড়া, কয়েকটি, পরীক্ষাকেন্দ্রে, ছাত্রলীগের,...","[no, no, no, no, no, no, no, no, no, no, no, n..."
4,ব্যাংকের চেয়ারম্যান ও ঢাকা বিশ্ববিদ্যালয়ের ই...,"[ব্যাংকের, চেয়ারম্যান, ও, ঢাকা, বিশ্ববিদ্যালয়ে...","[no, no, no, no, no, no, no, no, yes, yes, yes..."
5,তার হাতে ফাঁস হওয়া প্রশ্নের যে কাগজটি এসেছিল ...,"[তার, হাতে, ফাঁস, হওয়া, প্রশ্নের, যে, কাগজটি, ...","[no, no, no, no, no, no, no, no, no, no, no, n..."
6,"তিনি বলেন, 'বেলা আড়াইটার দিকেই পরীক্ষার কেন্দ...","[তিনি, বলেন, ,, ', বেলা, আড়াইটার, দিকেই, পরীক্...","[no, no, no, no, no, no, no, no, no, no, no, n..."
7,সেখান থেকে প্রশ্ন ফাঁস হতে পারে।',"[সেখান, থেকে, প্রশ্ন, ফাঁস, হতে, পারে, ।, ']","[no, no, no, no, no, no, no, no]"
8,"পরীক্ষা বাতিল করা হবে কি না, এমন প্রশ্নের জবাব...","[পরীক্ষা, বাতিল, করা, হবে, কি, না, ,, এমন, প্র...","[no, no, no, no, no, no, no, no, no, no, no, n..."
9,এ বিষয়ে পরিচালনা পর্ষদে আলোচনা করা হবে।,"[এ, বিষয়ে, পরিচালনা, পর্ষদে, আলোচনা, করা, হবে, ।]","[no, no, no, no, no, no, no, no]"


In [75]:
#savepath = "/content/drive/MyDrive/Hishab_assessment/Ds2_binary/"
#df3.to_csv(savepath+'ds2_binary_clean.csv',index=False)

### insight 2

* now I should try out my baseline model on this, to check whether it performs well or not

# Cleaning other datasets

* I think all data is the only other dataset I need to check
* it doesn't matter if I use one or two dataset, I will use both, and if I have time, I will ask them if I can use more dataset, to increase variety

## following the [repo ](https://github.com/Rifat1493/Bengali-NER/tree/master)

### prepro.py

In [76]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
import pandas as pd

def readfile(filename):
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,encoding="utf-8")
    sentences = []
    sentence = []
    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n" or line[1]=="\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue

        splits = line.split('\t')
        if (splits[-1]=='\n'):
            continue
        sentence.append([splits[0],splits[-1]])

    if len(sentence) >0:
        sentences.append(sentence)
        sentence = []
    return sentences


def createBatches(data):
    l = []
    for i in data:
        l.append(len(i[0]))
    l = set(l)
    batches = []
    batch_len = []
    z = 0
    for i in l:
        for batch in data:
            if len(batch[0]) == i:
                batches.append(batch)
                z += 1
        batch_len.append(z)
    return batches,batch_len



def createMatrices(sentences, word2Idx, label2Idx,char2Idx):
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']

    dataset = []

    wordCount = 0
    unknownWordCount = 0

    for sentence in sentences:
        wordIndices = []

        charIndices = []
        labelIndices = []

        for word,char,label in sentence:
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:

                try:
                   charIdx.append(char2Idx[x])
                except:
                   charIdx.append(char2Idx["UNKNOWN"])


            #Get the label and map to int
            wordIndices.append(wordIdx)

            charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])

        dataset.append([wordIndices, charIndices, labelIndices])

    return dataset

def iterate_minibatches(dataset,batch_len):
    start = 0
    for i in batch_len:
        tokens = []

        char = []
        labels = []
        data = dataset[start:i]
        start = i
        for dt in data:
            t,ch,l = dt
            l = np.expand_dims(l,-1)
            tokens.append(t)

            char.append(ch)
            labels.append(l)
        yield np.asarray(labels),np.asarray(tokens),np.asarray(char)

def addCharInformatioin(Sentences):
    for i,sentence in enumerate(Sentences):
        for j,data in enumerate(sentence):
            chars = [c for c in data[0]]
            Sentences[i][j] = [data[0],chars,data[1]]
    return Sentences

def padding(Sentences):
    maxlen = 101
    for sentence in Sentences:
        char = sentence[1]
        for x in char:
            maxlen = max(maxlen,len(str(x)))
    for i,sentence in enumerate(Sentences):
        Sentences[i][1] = pad_sequences(Sentences[i][1],101,padding='post')
    return Sentences

### validation.py

In [77]:
import numpy as np


#Method to compute the accruarcy. Call predict_labels to get the labels for the dataset
def compute_f1(predictions, correct, idx2Label):
    label_pred = []
    for sentence in predictions:
        label_pred.append([idx2Label[element] for element in sentence])

    label_correct = []
    for sentence in correct:
        label_correct.append([idx2Label[element] for element in sentence])


    #print label_pred
    #print label_correct

    prec = compute_precision(label_pred, label_correct)
    rec = compute_precision(label_correct, label_pred)

    f1 = 0
    if (rec+prec) > 0:
        f1 = 2.0 * prec * rec / (prec + rec);

    return prec, rec, f1

def compute_precision(guessed_sentences, correct_sentences):
    assert(len(guessed_sentences) == len(correct_sentences))
    correctCount = 0
    count = 0


    for sentenceIdx in range(len(guessed_sentences)):
        guessed = guessed_sentences[sentenceIdx]
        correct = correct_sentences[sentenceIdx]
        assert(len(guessed) == len(correct))
        idx = 0
        while idx < len(guessed):
            if guessed[idx][0] == 'B': #A new chunk starts
                count += 1

                if guessed[idx] == correct[idx]:
                    idx += 1
                    correctlyFound = True

                    while idx < len(guessed) and guessed[idx][0] == 'I': #Scan until it no longer starts with I
                        if guessed[idx] != correct[idx]:
                            correctlyFound = False

                        idx += 1

                    if idx < len(guessed):
                        if correct[idx][0] == 'I': #The chunk in correct was longer
                            correctlyFound = False


                    if correctlyFound:
                        correctCount += 1
                else:
                    idx += 1
            else:
                idx += 1

    precision = 0
    if count > 0:
        precision = float(correctCount) / count

    return precision

## reading the dataset

used this repo to get [initial processing](https://github.com/Rifat1493/Bengali-NER/tree/master) steps

In [78]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):
        tokens,char, labels = data
        tokens = np.asarray([tokens])

        char = np.asarray([char])
        pred = model.predict([tokens,char], verbose=False)[0]
        pred = pred.argmax(axis=-1) #Predict the classes
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

trainSentences = readfile("/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/Input/train_data.txt")
testSentences = readfile("/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/Input/test_data.txt")
#########
allsentences = readfile("/content/drive/MyDrive/Hishab_assessment/Bengali-NER-master/Bengali-NER-master/Input/all_data.txt")

#testSentences.pop(0)
#trainSentences.pop(0)
#trainSentences[0].pop(0)
#testSentences[0].pop(0)
allSentences = addCharInformatioin(allsentences)
trainSentences = addCharInformatioin(trainSentences)
testSentences = addCharInformatioin(testSentences)

In [79]:
"""for sentence in trainSentences:
    for token,char,label in sentence:
        labelSet.add(label)
        words[token.lower()] =True"""

labelSet = set()
words = {}

for dataset in [trainSentences,testSentences]:
    for sentence in dataset:

        for token,char,label in sentence:
          label = label[:-1]
          if label == 'TIM':
            label = 'O'
          labelSet.add(label)
          words[token.lower()] = label

In [80]:
labelSet

{'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O'}

In [81]:
words

{'\ufeffকী': 'O',
 'কারণে': 'O',
 'তাঁদের': 'O',
 'মধ্যে': 'O',
 'ঝামেলা': 'O',
 'হয়েছে': 'O',
 ',': 'O',
 'তা': 'O',
 'জানি': 'O',
 'না।’': 'O',
 'বরুড়া': 'B-LOC',
 'থানার': 'O',
 'ভারপ্রাপ্ত': 'O',
 'কর্মকর্তা': 'O',
 '(': 'O',
 'ওসি': 'O',
 ')': 'O',
 'এ': 'O',
 'কে': 'O',
 'এম': 'B-PER',
 'কাউছার': 'B-PER',
 'চৌধুরী': 'I-PER',
 'বলেন': 'O',
 '‘ঝলম': 'B-LOC',
 'ইউনিয়ন': 'O',
 'যুবলীগের': 'I-ORG',
 'কর্মী': 'O',
 'মাসুম': 'B-PER',
 'ছাত্রদলের': 'B-ORG',
 'সাদ্দামকে': 'B-PER',
 'গুলি': 'O',
 'করেন': 'O',
 '।': 'O',
 'আশঙ্কাজনক': 'O',
 'অবস্থায়': 'O',
 'উপজেলা': 'O',
 'স্বাস্থ্য': 'O',
 'কমপ্লেক্সে': 'O',
 'নেওয়ার': 'O',
 'পথেই': 'O',
 'তাঁর': 'O',
 'মৃত্যু': 'O',
 'হয়': 'O',
 'খুলনার': 'B-LOC',
 'দিঘলিয়া': 'B-ORG',
 'উপজেলার': 'O',
 'বারাকপুর': 'B-LOC',
 'মধ্যপাড়ায়': 'B-LOC',
 'গত': 'O',
 'সোমবার': 'O',
 'রাতে': 'O',
 'পুলিশ': 'O',
 'বাড়ি': 'O',
 'থেকে': 'O',
 'ডেকে': 'O',
 'কিছুক্ষণ': 'O',
 'পর': 'O',
 'কেসমত': 'I-PER',
 'আলী': 'B-PER',
 'মোল্লা': 'I-PER',
 '৫০': 'O',
 'নামে': 'O',
 

In [82]:
c = 0
# set korle sequence haraye jasse, so no set
for sentence in allSentences:
  for token,char,label in sentence:
    if token == '।':
      c +=1

# this is the number of sentences present
c

6581

In [83]:
# Extracting sentences
demoToken = []
demoTag   = []

finalTokens = []
finalTags   = []
yIdx = []
c=0
for sentence in allSentences:
  for token,char,label in sentence:
    label = label[:-1]
    if label == 'TIM':
      label = 'O'
    ####### extracting upto a sentence
    if token == '।':
      demoToken.append(token)
      demoTag.append(label)
      #print(demoToken)
      #print(demoTag)
      #print("\n")
      ##### appending tokens and tags of a sentence
      finalTokens.append(demoToken)
      finalTags.append(demoTag)
      ######### starting for new sentence
      demoToken = []
      demoTag   = []
      c+=1

    else:
      ####### if not end of a sentence keep adding tokens and tags
      demoToken.append(token)
      demoTag.append(label)

  #if c==5:
    #break

# so successfully separated all tokens and tags
print(c)
len(finalTokens),len(finalTags)# total number of sentences

6581


(6581, 6581)

In [84]:
a = finalTags[0]
a[0]

'O'

In [85]:
df5 = pd.DataFrame(columns=['tokens','tags'])
df5.tokens = finalTokens
df5.tags = finalTags
df5.head(20)

,tokens,tags
0,"[﻿উন্নয়নের, বিস্ময়, বাংলাদেশ, বাংলাদেশের, অগ্র...","[O, O, B-LOC, B-LOC, O, O, O, O, O]"
1,"[অর্থনীতি, ও, আর্থসামাজিক, বেশির, ভাগ, সূচকে, ...","[O, O, O, O, O, O, B-LOC, O, O, B-LOC, I-LOC, O]"
2,"[নিম্ন, আয়ের, দেশগুলোকে, ছাড়িয়েছে, তো, অনেক, আ...","[O, O, O, O, O, O, O, O]"
3,"[আন্তর্জাতিক, মুদ্রা, তহবিল, (, আইএমএফ, ), গত,...","[B-ORG, I-ORG, I-ORG, O, B-ORG, O, O, O, O, O,..."
4,"[সবাইকে, অন্তর্ভুক্ত, করে, প্রবৃদ্ধি, অর্জনের,...","[O, O, O, O, O, O, B-LOC, O, O, O, O, O, B-LOC..."
5,"[আবার, আরেকটি, আন্তর্জাতিক, দাতা, সংস্থা, বিশ্...","[O, O, B-ORG, I-ORG, I-ORG, B-ORG, O, O, O, O,..."
6,"[তবে, অন্তর্ভুক্তিমূলক, প্রবৃদ্ধি, অর্জনে, বাং...","[O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O]"
7,"[অন্তর্ভুক্তিমূলক, রাজনীতির, অভাবে, অর্থনীতির,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,"[অচল, হয়ে, পড়ছে, দেশের, অর্থনীতি, ,, কমছে, প্র...","[O, O, O, B-LOC, O, O, O, O, O]"
9,"[ফলে, সামাজিক, সূচকগুলোও, হুমকির, মধ্যে, পড়ে, ...","[O, O, O, O, O, O, O, O]"


In [86]:
# saving it to a csvFiles
#savepath = "/content/drive/MyDrive/Hishab_assessment/Bangla_NER_multiclass/"
#df5.to_csv(savepath+'bangla_ner_multiclass.csv',index=False)

# Merge both dataset

## MultiClass

In [87]:
import pandas as pd
dataset_path = "/content/drive/MyDrive/Hishab_assessment/Bangla_NER_multiclass/bangla_ner_multiclass.csv"
df1 = pd.read_csv(dataset_path,index_col=False)
print(len(df1))
df1.head(5)

6581


,tokens,tags
0,"['\ufeffউন্নয়নের', 'বিস্ময়', 'বাংলাদেশ', 'বাংল...","['O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O..."
1,"['অর্থনীতি', 'ও', 'আর্থসামাজিক', 'বেশির', 'ভাগ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '..."
2,"['নিম্ন', 'আয়ের', 'দেশগুলোকে', 'ছাড়িয়েছে', 'তো...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"
3,"['আন্তর্জাতিক', 'মুদ্রা', 'তহবিল', '(', 'আইএমএ...","['B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-ORG', 'O',..."
4,"['সবাইকে', 'অন্তর্ভুক্ত', 'করে', 'প্রবৃদ্ধি', ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '..."


In [89]:
dataset_path = "/content/drive/MyDrive/Hishab_assessment/Ds2_multiclass/ds2_multiclass_clean.csv"
df2 = pd.read_csv(dataset_path,index_col=False)
df2 = df2.drop('sentences', axis=1)
print(len(df2))
df2.head(5)

3345


,tokens,tags
0,"['অগ্রণী', 'ব্যাংকের', 'জ্যেষ্ঠ', 'কর্মকর্তা',...","['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O..."
1,"['ব্যাংকের', 'চেয়ারম্যানও', 'এ', 'অভিযোগের', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"
2,"['আজ', 'শুক্রবার', 'বিকেল', 'চারটা', 'থেকে', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['এ', 'ছাড়া', 'কয়েকটি', 'পরীক্ষাকেন্দ্রে', 'ছা...","['O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', '..."
4,"['ব্যাংকের', 'চেয়ারম্যান', 'ও', 'ঢাকা', 'বিশ্ব...","['O', 'O', 'O', 'B-ORG', 'I-ORG', 'B-ORG', 'I-..."


In [90]:
concatenated_df = pd.concat([df1, df2])
print(len(concatenated_df))
concatenated_df.head(5)

9926


,tokens,tags
0,"['\ufeffউন্নয়নের', 'বিস্ময়', 'বাংলাদেশ', 'বাংল...","['O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O..."
1,"['অর্থনীতি', 'ও', 'আর্থসামাজিক', 'বেশির', 'ভাগ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '..."
2,"['নিম্ন', 'আয়ের', 'দেশগুলোকে', 'ছাড়িয়েছে', 'তো...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"
3,"['আন্তর্জাতিক', 'মুদ্রা', 'তহবিল', '(', 'আইএমএ...","['B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-ORG', 'O',..."
4,"['সবাইকে', 'অন্তর্ভুক্ত', 'করে', 'প্রবৃদ্ধি', ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '..."


In [91]:
######### saving it to a csvFiles
#savepath = "/content/drive/MyDrive/Hishab_assessment/merged_ds/"
#concatenated_df.to_csv(savepath+'concatenated_bangla_ner_multiclass.csv',index=False)
#savepath+'concatenated_bangla_ner_multiclass.csv'